In [10]:
base_wiki_url = "https://overwatch.fandom.com/wiki/"

# Too lazy to scrap the page, just copy-pasted the hero names from here:
active_hero_list = [
    "Baptiste‎",
    "Bastion‎",
    "Brigitte‎",
    "Cassidy‎",
    "D.Va‎",
    "Doomfist‎",
    "Echo‎",
    "Genji‎",
    "Hanzo‎",
    "Junkrat‎",
    "Lifeweaver‎",
    "Lúcio‎",
    "Mei‎",
    "Mercy‎",
    "Moira‎",
    "Orisa‎",
    "Pharah‎",
    "Ramattra‎",
    "Reaper‎",
    "Reinhardt‎",
    "Roadhog‎",
    "Sigma‎",
    "Soldier: 76‎",
    "Sombra‎",
    "Symmetra‎",
    "Torbjörn‎",
    "Tracer‎",
    "Widowmaker‎",
    "Winston‎",
    "Wrecking Ball‎",
    "Zarya‎",
    "Zenyatta‎",
]

In [28]:
import requests
from  bs4 import BeautifulSoup


def parse_summary_info_and_image(soup):
    ability_summary = list(soup.contents)[1]
    type_info_container = list(ability_summary)[0]
    type_info = list(type_info_container)[0]
    type_container = type_info.contents[0]
    if len(type_info) == 3:
        effect_container = type_info.contents[1]
        return {"Type": type_container.contents[1].text,
                "Effect Type": effect_container.contents[1].text}
    return {"Type": type_container.contents[1].text}
    
def parse_ability_body_row(attribute):
    if len(attribute.contents) == 2:
        return {attribute.contents[0].text:attribute.contents[1].text}
    return {}
    
def parse_ability_box(soup):
    ability_data = {}
    # Parse main sections of ability box
    ability_header = soup.find("div", {"class": "abilityHeader"})
    ability_type = parse_summary_info_and_image(soup.find("div", {"class": "summaryInfoAndImage"}))
    ability_body = list(soup.contents)[2]
    # Parse ability name
    ability_name = ability_header.text

    # Parse ability keybind
    keybind = ability_header.find("span", {"title": "Hotkey"})
    if keybind:
        if keybind.text:
            keybind = keybind.text
        else:
            keybind = keybind.find("img")['alt']    
        ability_data = {"keybind": keybind}    
        ability_name = ability_header.text.replace(keybind, "")
    
    # Parse attributes in ability body
    ability_data.update(ability_type)
    for attribute in ability_body.contents:
        attribute_data = parse_ability_body_row(attribute)
        ability_data.update(attribute_data)
    # TODO: Parse the rest of the ability_box

    data = {ability_name: ability_data}
    return data

def get_abilities(character: str):
    # Generate hero-specific URL
    url = base_wiki_url + character.replace(" ", "_")
    print(f"Loading URL: {url}")
    # Query and parse URL
    r = requests.get(url)
    soup = BeautifulSoup(r.text)
    # content_soup = soup.find("div", {"class": "mw-parser-output"})
    ability_box_list = soup.find_all("div", {"class": "ability_box"})
    ability_data = {}
    for ability_box in ability_box_list:
        ability_data.update(parse_ability_box(ability_box))
    return ability_data

get_abilities("Reinhardt")
# for hero in active_hero_list:
#     get_abilities(hero)

Loading URL: https://overwatch.fandom.com/wiki/Reinhardt


{'Rocket Hammer': {'keybind': 'Key mouse 1',
  'Type': 'Weapon',
  'Effect Type': ' Melee',
  'Damage:': ' 85',
  'Headshot:': ' ✕',
  'Rate of fire:': ' 1.04 shots/s',
  'Cast time:': ' 0.4 + 0.5 seconds',
  'Duration:': ' 0.208 seconds (hitbox active)',
  'Max. range:': ' 5 meters'},
 'Barrier Field': {'keybind': 'Key mouse 2',
  'Type': 'Ability',
  'Health:': ' 1400 barrier health',
  'Cast time:': ' 0.1 + 0.29 seconds',
  'Area of effect:': ' Height: 4.250 metersWidth: 6.800 meters.',
  'Move. speed:': ' -30% penalty'},
 'Charge': {'keybind': 'LSHIFT',
  'Type': 'Ability',
  'Effect Type': ' Melee Movement',
  'Damage:': ' 225 (pinned to wall)50 (collision w/out pin)',
  'Cast time:': ' 0.6 + 0.5 seconds (wall impact)0.6 + 0.1 seconds (canceled)',
  'Duration:': ' 3 seconds (charge, max)0.3 seconds (charge, min)0.65 seconds (stun on wall hit)',
  'Max. range:': ' 50.2 meters',
  'Move. speed:': ' +200% buff (charge)31.2 m/s (knockback, indirect hit)11 m/s (knockback, canceled pin)